In [ ]:
!pip install groq
!pip install faiss-cpu
!pip install gpt4all
!pip install langchain_community
!pip install langchain_groq
!pip install langchain
!pip install sentence-transformers
!pip install langchain_openai
!pip install langchain-community

In [ ]:
import json
import networkx as nx
from groq import Groq
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.embeddings import GPT4AllEmbeddings
import openai
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
import time
# embeddings using langchain
from langchain.embeddings import SentenceTransformerEmbeddings

## Reading graph database

In [ ]:
file_path = 'cybersecurity-json-data.json'
with open(file_path, 'r') as file:
    data = [json.loads(line) for line in file]

## Encoding graph

In [ ]:
# Create a graph using NetworkX
graph = nx.Graph()

# Adding nodes and edges into graph
for item in data:
    if item['type'] == 'node':
        node_id = item['id']
        node_labels = item['labels']
        node_properties = item['properties']
        graph.add_node(node_id, labels=node_labels, **node_properties)
    elif item['type'] == 'relationship':
        source = item['start']['id']
        target = item['end']['id']
        edge_properties = item.get('properties', {})
        graph.add_edge(source, target, id=item['id'], label=item['label'], **edge_properties)

In [ ]:
# The function creates a string describing the node and its attributes
def create_node_string(graph):
    node_descriptions = []
    for node, props in graph.nodes(data=True):
        labels = props.pop('labels', [])
        labels_str = ', '.join(labels)
        prop_desc = ', '.join([f"{key}: {value}" for key, value in props.items()])
        node_descriptions.append(f"{node} [{labels_str}] ({prop_desc})")
    return ', '.join(node_descriptions)


In [ ]:
# Encoding graph function
def encode_graph_incident(graph):
    nodes_string = create_node_string(graph)
    output = "G describes a graph among nodes: \n%s.\n" % nodes_string
    if graph.edges():
        output += "In this graph:\n"
    for source_node in graph.nodes():
        target_nodes = list(graph.neighbors(source_node))
        target_nodes_str = ""
        nedges = 0
        for target_node in target_nodes:
            edge_props = graph.get_edge_data(source_node, target_node)
            edge_props_str = ', '.join([f"{key}: {value}" for key, value in edge_props.items()])
            target_nodes_str += f"{target_node} ({edge_props_str}), "
            nedges += 1
        if nedges > 1:
            output += "Node %s is connected to nodes %s.\n" % (
                source_node,
                target_nodes_str[:-2],
            )
        elif nedges == 1:
            output += "Node %s is connected to node %s.\n" % (
                source_node,
                target_nodes_str[:-2],
            )
    return output

# Encoding graph function
encoded_graph_incident = encode_graph_incident(graph)

In [ ]:
print(encoded_graph_incident)

## RAG

### Embedding

In [ ]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
embedding_model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}
embeddings = GPT4AllEmbeddings(
  model_name = embedding_model_name,
  gpt4all_kwargs = gpt4all_kwargs
)

### Vector db

In [ ]:
vector_db_path = "vectorstores/db_faiss"

def create_db_from_graph(encoded_graph):

    # Split graph data into smail text
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=8192,
        chunk_overlap=100,
        length_function=len

    )

    chunks = text_splitter.split_text(encoded_graph)


    #  Faiss Vector DB
    db = FAISS.from_texts(texts=chunks, embedding=embeddings)
    db.save_local(vector_db_path)
    return db


In [ ]:
db_test = create_db_from_graph(encoded_graph_incident)

### LLMs

In [ ]:
vector_db_path = "vectorstores/db_faiss"

# Load LLM
def load_llm(model_file):
    llm = CTransformers(
        model=model_file,
        model_type="llama",
        max_new_tokens=8192,
        temperature=0.01
    )
    return llm


### Create prompt template

In [ ]:
def creat_prompt(template):
    prompt = PromptTemplate(template = template, input_variables=["context", "question"])
    return prompt

### Create chain

In [ ]:
# Create chain
def create_chain(prompt, llm, db):
    llm_chain = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type= "stuff",
        retriever = db.as_retriever(search_kwargs = {"k":3}, max_tokens_limit=8192),
        return_source_documents = False,
        chain_type_kwargs= {'prompt': prompt}

    )
    return llm_chain

### Reading vector db

In [ ]:
def read_vectors_db():
    # Embeding
    db = FAISS.load_local(vector_db_path,
                          embedding_model_name,
                          allow_dangerous_deserialization=True)
    return db

### Start

In [ ]:
# start
db = read_vectors_db()

GROQ_API_KEY = "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"
llm_llama = ChatGroq(groq_api_key=GROQ_API_KEY, model="llama3-70b-8192")
llm_mixtral = ChatGroq(groq_api_key=GROQ_API_KEY, model="mixtral-8x7b-32768")

### Prompting template

In [ ]:
template = """<|im_start|>system\nUse the following information
                          to answer the question. Please exactly
                          \n
    {context}<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant"""
prompt = creat_prompt(template)


### LLM chain

In [ ]:
llm_chain_llama  = create_chain(prompt, llm_llama, db_test)
llm_chain_mixtral = create_chain(prompt, llm_mixtral, db_test)

## Zero - shot prompt

In [ ]:
question_zero =  """
Based on the following graph properties, generate detailed consistency rules (graph functional dependency and graph entity dependency).
Consider the structure, node information and relationships in the graph, and provide a set of rules that can be applied to maintain consistent
and accurate data.

For each consistency rule you identify, provide a clear description of the rule and generated the corresponding Cypher query to check the
number of nodes or relationships that satisfy the rule.
Your query should return the count of entities (nodes or relationships) that match the rule described.
Provide the query in the format of valid Cypher syntax, simple and ready for execution in a Neo4j databaseBelow is the input data:
Graph Information:

- Nodes: ["Group", "HighValue"], ["Group"], ["Domain", "HighValue"], ["GPO"], ["OU"], ["Computer"], ["User"].
- Relationships: GENERIC_ALL", "WRITE_OWNER", "WRITE_DACL", DC_SYNC", "GET_CHANGES", "GET_CHANGES_ALL", "ADMIN_TO", "OWNS", "GP_LINK", "CONTAINS", "HAS_SESSION", "MEMBER_OF", "EXECUTE_DCOM", "CAN_RDP", "ALLOWED_TO_DELEGATE", "GENERIC_WRITE".
- Properties: ["Group", "HighValue"].name, ["Group", "HighValue"].domain, ["Group", "HighValue"].objectid,  ["Group", "HighValue"].neo4jImportId, ["Group", "HighValue"].highvalue, ["Group"].name, ["Group"].domain, ["Group"].objectid, ["Group"].neo4jImportId,  ["Domain", "HighValue"].name, ["Domain", "HighValue"].domain,  ["Domain", "HighValue"],neo4jImportId,  ["Domain", "HighValue"].highvalue,  ["GPO"].name,  ["GPO"].neo4jImportId,  ["GPO"].domain,  ["GPO"].objectid,  ["OU"].blocksInheritance,  ["OU"].name,  ["OU"].neo4jImportId, ["OU"].domain,  ["OU"].objectid,  ["Computer"].operatingsystem, ["Computer"].name,  ["Computer"].domain, ["Computer"].objectid, ["Computer"].neo4jImportId, ["Computer"].owned,  ["Computer"].enabled, ["User"].displayname,  ["User"].owned,  ["User"].enabled,  ["User"].pwdlastset, ["User"].neo4jImportId, ["User"].domain, ["User"].lastlogon, ["User"].name, ["User"].objectid,  ["User"].hasspn,  ["Group"].enabled,  ["Group"].pwdlastset, ["Group"]displayname, ["Group"].owned, ["Group"].lastlogon, GENERIC_ALL.isacl, WRITE_OWNER.sacl,  WRITE_DACL.isacl, DC_SYNC.isacl,  GET_CHANGES.isacl,  GET_CHANGES_ALL.isacl, OWNS.isacl,  CONTAINS.isacl,  GP_LINK.enforced,  GP_LINK.isacl,
 GENERIC_WRITE.isacl.
"""

### LLAMA 3

In [ ]:
import time
time_start = time.time()
responses_llama_zero = llm_chain_llama.invoke({"query": question_zero})
time_end = time.time()
execution_time = time_end-time_start
print(f"Execution time: {execution_time}")
for response in str(responses_llama_zero).split("\n"):
  print(response)

### Mixtral

In [ ]:
import time
time_start_zr_mixtral = time.time()
responses_mixtral_zero = llm_chain_mixtral.invoke({"query": question_zero})
time_end_zr_mixtral = time.time()
execution_time_zr_mixtral = time_end_zr_mixtral - time_start_zr_mixtral
print(f"Execution time: {execution_time_zr_mixtral}")
for response in str(responses_llama_zero).split("\n"):
  print(response)

## Few - shot prompt

In [ ]:
few_shot_prompt = """
    Examples of consistency Rules:

    1. Unique `neo4jImportId` per node: Each node should have a unique `neo4jImportId` property
    2. Unique `objectid` per node: Each node should have a unique `objectid` property.
    3. Only allowed node labels are `User`, `Group`, `Domain`, `OU`, `GPO`, and `Computer`

    Task: Generate new rules to ensure consistency and accuracy in the graph database, considering all node types and relationships.
    For each consistency rule you identify, provide a clear description of the rule and generated the corresponding Cypher query to check the
    number of nodes or relationships that satisfy the rule.
    Your query should return the count of entities (nodes or relationships) that match the rule described.
    Provide the query in the format of valid Cypher syntax, simple and ready for execution in a Neo4j databaseBelow is the input data:
    Graph Information:

    - Nodes: ["Group", "HighValue"], ["Group"], ["Domain", "HighValue"], ["GPO"], ["OU"], ["Computer"], ["User"].
    - Relationships: GENERIC_ALL", "WRITE_OWNER", "WRITE_DACL", DC_SYNC", "GET_CHANGES", "GET_CHANGES_ALL", "ADMIN_TO", "OWNS", "GP_LINK", "CONTAINS", "HAS_SESSION", "MEMBER_OF", "EXECUTE_DCOM", "CAN_RDP", "ALLOWED_TO_DELEGATE", "GENERIC_WRITE".
    - Properties: ["Group", "HighValue"].name, ["Group", "HighValue"].domain, ["Group", "HighValue"].objectid,  ["Group", "HighValue"].neo4jImportId, ["Group", "HighValue"].highvalue, ["Group"].name, ["Group"].domain, ["Group"].objectid, ["Group"].neo4jImportId,  ["Domain", "HighValue"].name, ["Domain", "HighValue"].domain,  ["Domain", "HighValue"],neo4jImportId,  ["Domain", "HighValue"].highvalue,  ["GPO"].name,  ["GPO"].neo4jImportId,  ["GPO"].domain,  ["GPO"].objectid,  ["OU"].blocksInheritance,  ["OU"].name,  ["OU"].neo4jImportId, ["OU"].domain,  ["OU"].objectid,  ["Computer"].operatingsystem, ["Computer"].name,  ["Computer"].domain, ["Computer"].objectid, ["Computer"].neo4jImportId, ["Computer"].owned,  ["Computer"].enabled, ["User"].displayname,  ["User"].owned,  ["User"].enabled,  ["User"].pwdlastset, ["User"].neo4jImportId, ["User"].domain, ["User"].lastlogon, ["User"].name, ["User"].objectid,  ["User"].hasspn,  ["Group"].enabled,  ["Group"].pwdlastset, ["Group"]displayname, ["Group"].owned, ["Group"].lastlogon, GENERIC_ALL.isacl, WRITE_OWNER.sacl,  WRITE_DACL.isacl, DC_SYNC.isacl,  GET_CHANGES.isacl,  GET_CHANGES_ALL.isacl, OWNS.isacl,  CONTAINS.isacl,  GP_LINK.enforced,  GP_LINK.isacl,
     GENERIC_WRITE.isacl.
"""

### LLAMA 3

In [ ]:
time_start_fs_llama = time.time()
responses_llama_few = llm_chain_llama.invoke({"query": question_zero})
time_end_fs_llama = time.time()
execution_time_fs_llama = time_end_fs_llama - time_start_fs_llama
print(f"Execution time: {execution_time_fs_llama}")
for response in str(responses_llama_few).split("\n"):
  print(response)

### Mixtral

In [ ]:
time_start_fs_mixtral = time.time()
responses_mixtral_few = llm_chain_llama.invoke({"query": question_zero})
time_end_fs_mixtral = time.time()
execution_time_fs_mixtral = time_end_fs_mixtral - time_start_fs_mixtral
print(f"Execution time: {execution_time_fs_mixtral}")
for response in str(responses_mixtral_few).split("\n"):
  print(response)